# Let's load a dataset and try training!

In [32]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from pathlib import Path
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import sys

sys.path.append('../')
from scripts import forward_process as fp, reverse_process as rp, train, util
from scripts import customUNet as un

In [12]:
ab1 = np.load("../data/image-colorization/ab/ab/ab1.npy")
ab2 = np.load("../data/image-colorization/ab/ab/ab2.npy")
ab3 = np.load("../data/image-colorization/ab/ab/ab3.npy")
l = np.load("../data/image-colorization/l/gray_scale.npy")

ab = np.concatenate((ab1,ab2,ab3),axis=0)

def createLab(n=100):
    lab = np.zeros((n,224,224,3))
    
    lab[:,:,:,0] = l[0:n:]
    lab[:,:,:,1:] = ab[0:n:]    
    
    return lab.astype("uint8")

## Let's load 1000 images

In [49]:
lab = createLab(1000)

# Resize to 64x64 for lower computational load
resized = []
for i in lab:
    img = cv2.normalize(i, None, alpha = -1, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    resized.append(cv2.resize(img, dsize=(64, 64), interpolation=cv2.INTER_CUBIC))

# Convert to tensor
lab = torch.tensor(np.array(resized))
dataloader = DataLoader(lab, batch_size=25, shuffle=True)
print(len(lab))

1000


TODO:

1. Define custom DataLoader: \
    * \__init__ \
    * \__len__ \
    * \__getitem__ \

See https://www.kaggle.com/code/varunnagpalspyz/pix2pix-is-all-you-need#%F0%9F%A7%B0-Utility-Functions \

2. Choose valid data

3. Load training data, valid data

4. Train!

In [28]:
def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

results_folder = Path("../results")
results_folder.mkdir(exist_ok = True)
save_and_sample_every = 100

In [47]:
model = un.CustomConditionalUNet()
device = util.set_device()
model.to(device)
x = sum(p.numel() for p in model.parameters())
print(x)
optimizer = Adam(model.parameters(), lr=1e-3)

110075811


In [42]:
diffTerms = util.ConstantDiffusionTerms(1000, fp.linear_beta_schedule)
train.train(5,device,optimizer,dataloader,50,1000, diffTerms,model)

# from torchvision.utils import save_image

# epochs = 5

# for epoch in range(epochs):
#     for step, batch in enumerate(dataloader):
#         optimizer.zero_grad()

#         batch_size = batch.shape[0]
#         batch = batch.to(util.set_device())

#       # Algorithm 1 line 3: sample t uniformally for every example in the batch
#         t = torch.randint(0, 1000, (batch_size,), device=device).long()

#         loss = train.p_loss(model, batch, t, loss_type="huber")

#         if step % 100 == 0:
#             print("Loss:", loss.item())

#         loss.backward()
#         optimizer.step()

#       # save generated images
#         if step != 0 and step % save_and_sample_every == 0:
#             milestone = step // save_and_sample_every
#             batches = num_to_groups(4, batch_size)
#             all_images_list = list(map(lambda n: sample(model, batch_size=n, channels=channels), batches))
#             all_images = torch.cat(all_images_list, dim=0)
#             all_images = (all_images + 1) * 0.5
#             save_image(all_images, str(results_folder / f'sample-{milestone}.png'), nrow = 6)

ValueError: too many values to unpack (expected 2)